NOTE: This Notebook was used in Google Colab

In [1]:
# install libs if required
!pip install optuna
!pip install sentence_transformers

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/15/da/68883911855d8b4d521f9a370e4e6aab8232b91c1d8d5a8348c4680c6642/optuna-3.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/df/ed/c884465c33c25451e4a5cd4acad154c29e5341e3214e220e7f3478aa4b0d/alembic-1.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=1.3.0 from https://files.pythonhosted.org/packages/67/cb/c1d08c7769ccd3c33078c39ea92eda2e6864dbfb6a9e2dfff8b812038972/SQLAlchemy-2.0.31-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/03/62/70f5a0c2dd208f9f3f2f9afd103aec42ee4d9ad2401d78342f75e9b8da36/Mako-1.3.

In [ ]:
# auto reload modules
%load_ext autoreload
%autoreload 0

In [ ]:
# colab specific remove if not in Colab and fix paths
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set data paths
path_to_data = '/content/drive/MyDrive/civol/iceccme/'
filepath_data = path_to_data + 'label_final.csv'

# output path
modelpath = path_to_data + 'models_own/'

In [ ]:
 # helper function to get data in required format
def get_dataset_from_df(col_text_a, col_text_b, label_col, df):
    dataset = []
    for index, row in df.iterrows():
        text_a = row[col_text_a]
        text_b = row[col_text_b]
        label = row[label_col]
        dataset.append((text_a, text_b, label))
        dataset.append((text_b, text_a, label))
    return dataset

In [ ]:
# imports
import logging
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
import torch

<ipython-input-6-8dfc5c26fce2>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


tqdm.notebook.tqdm_notebook

In [ ]:
# check for gpu available
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# read in data
df_base = pd.read_csv(filepath_data, sep=';')
df_base['title_description'] = df_base['title'] + ': ' + df_base['description']

In [ ]:
# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# get Data in required format
dataset = get_dataset_from_df('goal', 'title_description', 'label_manual', df_base)

In [ ]:
# logging callback function to store and print current loss
current_loss = None


def simple_callback(loss, epoch, steps):
    global current_loss
    current_loss = loss
    logging.log(logging.INFO, f"Loss: {loss}, epoch: {epoch}, steps: {steps}")

In [ ]:
# Define the objective function for Optuna's hyperparameter optimization.
def objective(trial):
    try:
        # Log the start of a trial.
        logging.log(logging.INFO, f"Trial {trial.number}")
        logging.log(logging.INFO, f"Model created")

        # Initialize the CrossEncoder model using the pre-trained 'cross-encoder/stsb-roberta-base'.
        model = CrossEncoder('cross-encoder/stsb-roberta-base', num_labels=1, device=device)

        # Suggest hyperparameters to be optimized.
        learning_rate = trial.suggest_float("learning_rate", 5e-5, 1e-1,
                                            log=True)  # Suggest a learning rate between 5e-5 and 1e-1 on a log scale.
        batch_size = trial.suggest_int("batch_size", 2, 16)  # Suggest an integer batch size between 2 and 16.
        num_epochs = trial.suggest_int("num_epochs", 1, 20)  # Suggest an integer number of epochs between 1 and 20.
        warmup_steps = trial.suggest_int("warmup_steps", 0,
                                         1000)  # Suggest an integer number of warmup steps between 0 and 1000.
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1,
                                           log=True)  # Suggest a weight decay value between 1e-5 and 1e-1 on a log scale.
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)  # Suggest a dropout rate between 0.1 and 0.5.

        # Set the dropout rate for the model.
        model.dropout_rate = dropout_rate

        # Log the chosen hyperparameters.
        logging.log(logging.INFO,
                    f"Learning rate: {learning_rate}, batch size: {batch_size}, num epochs: {num_epochs}, warmup steps: {warmup_steps}, weight decay: {weight_decay}, dropout rate: {dropout_rate}")

        # Split the dataset into training and validation sets.
        train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=347657)

        # Create data loaders for the training and validation data.
        train_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in train_data], shuffle=True,
                                      batch_size=batch_size)
        valid_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in val_data],
                                      batch_size=batch_size)

        # Prepare the validation data for the evaluator.
        val_sentence_pairs = [[t[0], t[1]] for t in val_data]
        labels = [t[2] for t in val_data]

        # Initialize the evaluator for binary classification.
        evaluator = CEBinaryClassificationEvaluator(sentence_pairs=val_sentence_pairs, labels=labels, name='goal_eval',
                                                    write_csv=True, show_progress_bar=True)

        # Fit the model using the specified hyperparameters and data loaders.
        model.fit(train_dataloader=train_dataloader,
                  epochs=num_epochs,
                  warmup_steps=warmup_steps,
                  output_path=modelpath + 'model_trial_' + str(trial.number),
                  optimizer_params={'lr': learning_rate, 'weight_decay': weight_decay},
                  show_progress_bar=True,
                  evaluator=evaluator,
                  evaluation_steps=1000,
                  save_best_model=True,
                  callback=simple_callback
                  )
        logging.log(logging.INFO, f"Model fitted")

        # Evaluate the model and log the accuracy.
        accuracy = evaluator(model, output_path=modelpath + 'model_trial_' + str(trial.number))
        logging.log(logging.INFO, f"Accuracy: {accuracy}")

        # Report the current loss to Optuna for pruning purposes.
        global current_loss
        if current_loss:
            trial.report(current_loss, trial.number)
            logging.log(logging.INFO, f"Trial pruned")
            if trial.should_prune():
                raise optuna.TrialPruned()

    except Exception as e:
        # Log any errors that occur during the trial.
        logging.log(logging.ERROR, f"Error: {e}")
        raise e
    finally:
        # Clean up the model and clear the CUDA cache.
        if model:
            del model
        if device == 'cuda':
            torch.cuda.empty_cache()
        elif device == 'mps':
            torch.mps.empty_cache()

    # Return the accuracy as the result of the trial if it exists.
    if accuracy:
        return accuracy


In [ ]:
# Define path of database
sql_path = "sqlite:///" + modelpath + "db_max_own.sqlite3"

# create study
study = optuna.create_study(storage=sql_path, study_name="own_data_hpo_max_3_d_wd", load_if_exists=True,
                            direction="maximize")

# run hyperparameter optimization
study.optimize(objective, n_trials=12)

[I 2024-06-28 13:21:27,713] Using an existing study with name 'own_data_f1' instead of creating a new one.
2024-06-28 13:21:27,776 - INFO - Trial 12
2024-06-28 13:21:27,777 - INFO - Model created
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

2024-06-28 15:36:48,141 - INFO - CEBinaryClassificationEvaluator: Evaluating the model on goal_eval dataset after epoch 6:


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-06-28 15:36:52,562 - INFO - Accuracy:           80.99	(Threshold: 0.0198)
2024-06-28 15:36:52,563 - INFO - F1:                 78.90	(Threshold: 0.0198)
2024-06-28 15:36:52,564 - INFO - Precision:          76.79
2024-06-28 15:36:52,565 - INFO - Recall:             81.13
2024-06-28 15:36:52,567 - INFO - Average Precision:  76.79

2024-06-28 15:36:52,573 - INFO - Loss: 0.7679267492898271, epoch: 6, steps: -1
2024-06-28 15:36:52,574 - INFO - Save model to /content/drive/MyDrive/civol/iceccme/models_own/model_trial_f1_23
2024-06-28 15:36:54,357 - INFO - Model fitted
2024-06-28 15:36:54,359 - INFO - CEBinaryClassificationEvaluator: Evaluating the model on goal_eval dataset:


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-06-28 15:36:58,487 - INFO - Accuracy:           80.99	(Threshold: 0.0198)
2024-06-28 15:36:58,488 - INFO - F1:                 78.90	(Threshold: 0.0198)
2024-06-28 15:36:58,490 - INFO - Precision:          76.79
2024-06-28 15:36:58,491 - INFO - Recall:             81.13
2024-06-28 15:36:58,492 - INFO - Average Precision:  76.79

2024-06-28 15:36:58,497 - INFO - -Accuracy: 0.7679267492898271
2024-06-28 15:36:58,646 - INFO - -F1: 0.7889908256880735 of trial 23
[I 2024-06-28 15:36:59,048] Trial 23 finished with value: 0.7889908256880735 and parameters: {'learning_rate': 6.425844041348337e-05, 'batch_size': 7, 'num_epochs': 7, 'warmup_steps': 74, 'weight_decay': 5.340595128915974e-06, 'dropout_rate': 0.4994447285759936}. Best is trial 6 with value: 0.847457627118644.
